# DS Lab - Final Assignment: Robust Journey Planning #

## Load the data

In [2]:
#Importations.
import pandas as pd
import glob
from geopy import distance
import numpy as np

### Load the data published by the Open Data Platform Swiss Public Transport (https://opentransportdata.swiss).



In [3]:
#path on my computer of the files.
path = "2017-09/2017-09-13istdaten.csv"

In [4]:
#useful list
data = []
#read each csv we had.
for fname in sorted(glob.glob(path)):
    d = pd.read_csv(fname, sep=';',low_memory=False)
    data.append(d)
    
#process the data from the csv files to a dataframe.
data = pd.concat(data)
data = pd.DataFrame(data)
data = data.reset_index()
data = data.drop(['index'], axis = 1)

In [5]:
#check the tail.
data.tail()

,BETRIEBSTAG,FAHRT_BEZEICHNER,BETREIBER_ID,BETREIBER_ABK,BETREIBER_NAME,PRODUKT_ID,LINIEN_ID,LINIEN_TEXT,UMLAUF_ID,VERKEHRSMITTEL_TEXT,...,FAELLT_AUS_TF,BPUIC,HALTESTELLEN_NAME,ANKUNFTSZEIT,AN_PROGNOSE,AN_PROGNOSE_STATUS,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF
889269,13.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504294,Vuiteboeuf,13.09.2017 05:57,NaN,UNBEKANNT,13.09.2017 05:57,NaN,UNBEKANNT,False
889270,13.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504295,Baulmes,13.09.2017 06:01,NaN,UNBEKANNT,13.09.2017 06:01,NaN,UNBEKANNT,False
889271,13.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504296,Six-Fontaines,13.09.2017 06:05,NaN,UNBEKANNT,13.09.2017 06:05,NaN,UNBEKANNT,False
889272,13.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504297,Trois-Villes,13.09.2017 06:12,NaN,UNBEKANNT,13.09.2017 06:12,NaN,UNBEKANNT,False
889273,13.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504298,Ste-Croix,13.09.2017 06:21,NaN,UNBEKANNT,NaN,NaN,PROGNOSE,False


### Load the station list data.

In [6]:
#path.
station_path = "BFKOORD_GEO"

In [7]:
station_file  = open(station_path, "r")

In [8]:
BPUIC = []
long = []
lat = []
name = []

for line in station_file:
    l = line.split()
    BPUIC.append(l[0])
    long.append(l[1])
    lat.append(l[2])
    name.append(l[5])


In [9]:
d = {'BPUIC':BPUIC,
     'longitude' : long,
     'latitude' : lat,
     'station name': name}
station_df = pd.DataFrame(d)
station_df = station_df[['BPUIC','longitude','latitude']]
station_df['BPUIC'] = station_df['BPUIC'].astype(np.int64)

In [10]:
station_df.head()

,BPUIC,longitude,latitude
0,2,26.074412,44.446770
1,3,1.811446,50.901549
2,4,1.075329,51.284212
3,5,-3.543547,50.729172
4,7,9.733756,46.922368


In [11]:
zurich_coord = (47.407665036,8.540331172)
salamanca = (40.96, -5.50)

In [12]:
print(distance.distance(zurich_coord, salamanca).km)

1328.6915399458765


In [13]:
df = pd.merge(data,station_df,on='BPUIC',how='left')

In [14]:
df.head()

,BETRIEBSTAG,FAHRT_BEZEICHNER,BETREIBER_ID,BETREIBER_ABK,BETREIBER_NAME,PRODUKT_ID,LINIEN_ID,LINIEN_TEXT,UMLAUF_ID,VERKEHRSMITTEL_TEXT,...,HALTESTELLEN_NAME,ANKUNFTSZEIT,AN_PROGNOSE,AN_PROGNOSE_STATUS,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF,longitude,latitude
0,13.09.2017,80:06____:17010:000,80:06____,DB,DB Regio AG,Zug,17010,RE,NaN,RE,...,Basel Bad Bf,NaN,NaN,PROGNOSE,13.09.2017 05:45,13.09.2017 05:45:00,PROGNOSE,False,7.607284,47.568147
1,13.09.2017,80:06____:17012:000,80:06____,DB,DB Regio AG,Zug,17012,RE,NaN,RE,...,Basel Bad Bf,NaN,NaN,PROGNOSE,13.09.2017 06:34,13.09.2017 06:34:00,PROGNOSE,False,7.607284,47.568147
2,13.09.2017,80:06____:17013:000,80:06____,DB,DB Regio AG,Zug,17013,RE,NaN,RE,...,Basel Bad Bf,13.09.2017 06:25,13.09.2017 06:28:00,PROGNOSE,NaN,NaN,PROGNOSE,False,7.607284,47.568147
3,13.09.2017,80:06____:17014:000,80:06____,DB,DB Regio AG,Zug,17014,RE,NaN,RE,...,Basel Bad Bf,NaN,NaN,PROGNOSE,13.09.2017 09:48,13.09.2017 09:48:00,PROGNOSE,False,7.607284,47.568147
4,13.09.2017,80:06____:17015:000,80:06____,DB,DB Regio AG,Zug,17015,RE,NaN,RE,...,Basel Bad Bf,13.09.2017 08:06,13.09.2017 08:11:00,PROGNOSE,NaN,NaN,PROGNOSE,False,7.607284,47.568147


# Create Network

In [15]:
import networkx as nx
import datetime

In [16]:
# Renaming the columns of the dataframe
df.columns = ['trip_date', 'trip_id', 'operator_id', 'operator_abk', 
                'operator_name', 'product_id', 'line_id', 'line_text',
                'circulation_id', 'transport_text', 'additional_trip',
                'failed', 'stop_id', 'stop_name', 'scheduled_arrival_time',
                'actual_arrival_time', 'actual_arrival_time_status', 'scheduled_departure_time',
                'actual_departure_time',  'actual_departure_time_status', 'no_stop',
                'longitude', 'latitude']

In [17]:
# Convert columns 'trip_date', scheduled_arrival_time' and 'scheduled departure time' to datetime
df['trip_date'] = pd.to_datetime(df['trip_date'], format='%d.%m.%Y', errors='ignore')
df['scheduled_arrival_time'] = pd.to_datetime(df['scheduled_arrival_time'], format='%d.%m.%Y %H:%M', errors='ignore')
df['scheduled_departure_time'] = pd.to_datetime(df['scheduled_departure_time'], format='%d.%m.%Y %H:%M', errors='ignore')
df['actual_arrival_time'] = pd.to_datetime(df['actual_arrival_time'], format='%d.%m.%Y %H:%M:%S', errors='ignore')
df['actual_departure_time'] = pd.to_datetime(df['actual_departure_time'], format='%d.%m.%Y %H:%M:%S', errors='ignore')

In [18]:
info_trips = pd.DataFrame()
visu_trips = pd.DataFrame()

# id_ = df['trip_id'].unique()[555]
for id_ in df['trip_id'].unique()[400:600]:
    temp = df.loc[df['trip_id'] == id_]
    # We consider trips having at least 2 stops
    if len(temp) > 1:
        # For a given trip id sort arrival times at the stops
        temp = temp.sort_values(by=['scheduled_arrival_time'], axis='index', ascending=True, inplace=False, na_position='first')
        # Select only relevant columns in the dataframe
        temp = temp[['operator_name', 'product_id', 'line_id', 'scheduled_departure_time', 'actual_departure_time', 
                     'scheduled_arrival_time', 'actual_arrival_time', 'stop_name', 'longitude', 'latitude']]
        # Concatenate the result into a final DataFrame
        info_trips = pd.concat([info_trips, temp], axis=0, levels=['line_id', 'product_id'], ignore_index=False)

        # Create a visualization DataFrame showing bus stops, 
        # departure time and arrival time for each trip id
        visu = temp.copy()
        visu['stop_list'] = ' | '.join(visu['stop_name'])
        # Departure time at the start of the trip
        visu['scheduled_terminus_departure_time'] = visu['scheduled_departure_time'].iloc[0]
        # Actual departure time at the start of the trip
        visu['actual_terminus_departure_time'] = visu['actual_departure_time'].iloc[0]
        # Arrival time at the end of the trip
        visu['scheduled_terminus_arrival_time'] = visu['scheduled_arrival_time'].iloc[len(visu)-1]
        # Actual arrival time at the end of the trip
        visu['actual_terminus_arrival_time'] = visu['actual_arrival_time'].iloc[len(visu)-1]
        
        #visu = visu.reset_index(['operator_name', 'product_id', 'line_id'])
        # Select only relevant columns in the dataframe
        visu = visu[['operator_name', 'product_id', 'line_id', 'stop_list', 'scheduled_terminus_departure_time', 
                     'scheduled_terminus_arrival_time', 'actual_terminus_departure_time', 
                     'actual_terminus_arrival_time']].iloc[:1]
        # Concatenate the result into a final DataFrame
        visu_trips = pd.concat([visu_trips, visu], axis=0, ignore_index=False)

In [19]:
info_trips = info_trips.set_index(['operator_name', 'line_id', 'product_id']).sort_index(axis=0, level=['line_id'])
info_trips = info_trips[['stop_name', 'longitude', 'latitude', 'scheduled_departure_time', 'actual_departure_time',
                         'scheduled_arrival_time', 'actual_arrival_time']]

In [20]:
info_trips

stop_name  \
operator_name                   line_id product_id                       
Schweizerische Bundesbahnen SBB 11031   Zug           Lancy-Pont-Rouge   
                                        Zug                     Genève   
                                        Zug              Les Tuileries   
                                        Zug           Genthod-Bellevue   
                                        Zug           Creux-de-Genthod   
                                        Zug                    Versoix   
                                        Zug                 Pont-Céard   
                                        Zug                       Mies   
                                        Zug                     Tannay   
                                        Zug                     Coppet   
                                11032   Zug                     Coppet   
                                        Zug                     Tannay   
                                        Zug                       Mies   
                                        Zug                 Pont-Céard   
                                        Zug                    Versoix   
                                        Zug           Creux-de-Genthod   
                                        Zug           Genthod-Bellevue   
                                        Zug              Les Tuileries   
                                        Zug                     Genève   
                                        Zug           Lancy-Pont-Rouge   
                                11033   Zug           Lancy-Pont-Rouge   
                                        Zug                     Genève   
                                        Zug              Les Tuileries   
                                        Zug           Genthod-Bellevue   
                                        Zug           Creux-de-Genthod   
                                        Zug                    Versoix   
                                        Zug                 Pont-Céard   
                                        Zug                       Mies   
                                        Zug                     Tannay   
                                        Zug                     Coppet   
...                                                                ...   
                                12289   Zug                   Vallorbe   
                                        Zug                     Le Day   
                                        Zug               Bretonnières   
                                        Zug          Croy-Romainmôtier   
                                        Zug                      Arnex   
                                        Zug                  La Sarraz   
                                        Zug         Cossonay-Penthalaz   
                                        Zug          Vufflens-la-Ville   
                                        Zug                   Bussigny   
                                        Zug                  Renens VD   
                                        Zug              Prilly-Malley   
                                        Zug                   Lausanne   
                                        Zug                      Pully   
                                        Zug                      Lutry   
                                        Zug                Villette VD   
                                        Zug                      Cully   
                                        Zug                    Epesses   
                                        Zug                      Rivaz   
                                        Zug                St-Saphorin   
                                        Zug                      Vevey   
                                        Zug           La Tour-de-Peilz   
                                        Zug                     B

In [21]:
visu_trips = visu_trips.set_index(['operator_name', 'stop_list', 'product_id']).sort_index(axis=0, level=['stop_list'])
visu_trips = visu_trips[['line_id','scheduled_terminus_departure_time', 'scheduled_terminus_arrival_time', 
                         'actual_terminus_departure_time', 'actual_terminus_arrival_time']]

In [22]:
visu_trips.head()

line_id  \
operator_name                   stop_list                                          product_id           
Schweizerische Bundesbahnen SBB Chiasso | Lugano | Bellinzona | Arth-Goldau | Z... Zug             12   
                                Coppet | Tannay | Mies | Pont-Céard | Versoix |... Zug          11078   
                                Coppet | Tannay | Mies | Pont-Céard | Versoix |... Zug          11072   
                                                                                   Zug          11074   
                                                                                   Zug          11076   

                                                                                              scheduled_terminus_departure_time  \
operator_name                   stop_list                                          product_id                                     
Schweizerische Bundesbahnen SBB Chiasso | Lugano | Bellinzona | Arth-Goldau | Z... Zug                      2017-09-13 08:15:00   
                                Coppet | Tannay | Mies | Pont-Céard | Versoix |... Zug                      2017-09-13 20:33:00   
                                Coppet | Tannay | Mies | Pont-Céard | Versoix |... Zug                      2017-09-13 19:03:00   
                                                                                   Zug                      2017-09-13 19:33:00   
                                                                                   Zug                      2017-09-13 20:03:00   

                                                                                              scheduled_terminus_arrival_time  \
operator_name                   stop_list                                          product_id                                   
Schweizerische Bundesbahnen SBB Chiasso | Lugano | Bellinzona | Arth-Goldau | Z... Zug                    2017-09-13 10:51:00   
                                Coppet | Tannay | Mies | Pont-Céard | Versoix |... Zug                    2017-09-13 20:56:00   
                                Coppet | Tannay | Mies | Pont-Céard | Versoix |... Zug                    2017-09-13 19:34:00   
                                                                                   Zug                    2017-09-13 20:04:00   
                                                                                   Zug                    2017-09-13 20:34:00   

                                                                                              actual_terminus_departure_time  \
operator_name                   stop_list                                          product_id                                  
Schweizerische Bundesbahnen SBB Chiasso | Lugano | Bellinzona | Arth-Goldau | Z... Zug                   2017-09-13 08:21:51   
                                Coppet | Tannay | Mies | Pont-Céard | Versoix |... Zug                                   NaT   
                                Coppet | Tannay | Mies | Pont-Céard | Versoix |... Zug                   2017-09-13 19:05:24   
                                                                                   Zug                   2017-09-13 19:33:37   
                                                                                   Zug                                   NaT   

                                                                                              actual_terminus_arrival_time  
operator_name                   stop_list                                          product_id                               
Schweizerische Bundesbahnen SBB Chiasso | Lugano | Bellinzona | Arth-Goldau | Z... Zug                 2017-09-13 10:50:51  
                                Coppet | Tannay | Mies | Pont-Céard | Versoix |... Zug                                 NaT  
                                Coppet | Tannay | Mies | Pont-Céard | Versoix |... Zug                 2017-09-13 19:34:23  
              

# EDGES

In [23]:
# Create table that contains all edges in the network.
# An edge is essentially caracterized by its origin, destination and mean of transport (product_id)

# Initialize empty dataframe
partial_arcs_df = pd.DataFrame()
# Get all trips and their associated mean of transport from table 'visu_trips'
trips = set(visu_trips.index.get_values())
# Iterate on each trip to construct arcs
for _, stop_list, transport in trips:
    # list of stop names along the trip
    stop_names = stop_list.split(' | ')
    # create arcs associated with the trip
    trip_arcs = list(zip(stop_names[0:-1:1], stop_names[1::1]))
    # temporary dataframe
    temp = pd.DataFrame(trip_arcs, columns=['origin', 'destination'])
    temp['product_id'] = transport
    # concatenate new arcs with existing arcs in a single dataframe
    partial_arcs_df = pd.concat([partial_arcs_df, temp])

In [24]:
partial_arcs_df = partial_arcs_df.reset_index(drop=True)

In [25]:
# print the result
partial_arcs_df.head()

,origin,destination,product_id
0,Genève,Vernier,Zug
1,Vernier,Meyrin,Zug
2,Meyrin,Zimeysa,Zug
3,Zimeysa,Satigny,Zug
4,Satigny,Russin,Zug


In [68]:
df_partial_edges = pd.DataFrame()

for index, arc in partial_arcs_df.iterrows():
    origin = arc['origin']
    destination = arc['destination']
    #print(origin, destination)
    # temporary dataframe with all lines having either origin, destination or both in their stop list
    temp = info_trips[((info_trips['stop_name'] == origin) | (info_trips['stop_name'] == destination))]
    temp = temp.reset_index('line_id')
    # Get arcs
    for line_id in temp['line_id'].unique():
        # utility variable to find arcs in the dataframe 'info_trips'
        cond = temp[temp['line_id'] == line_id]
        # 1. A line must contain both stops included in the arc
        # 2. As stops are stored in the order they are visited, origin must be above destination
        if len(cond) == 2 and (cond.iloc[:1]['stop_name'] == origin).bool():
            #temp = temp.set_index(['line_id'])
            # compute scheduled arc departure time
            scheduled_departure_time = cond['scheduled_departure_time'].iloc[0]
            # compute scheduled arc arrival time
            scheduled_arrival_time = cond['scheduled_arrival_time'].iloc[1]
            # compute scheduled arc travel time
            scheduled_travel_time = scheduled_arrival_time - scheduled_departure_time
            
            # compute actual arc departure time
            actual_departure_time = cond['actual_departure_time'].iloc[0]
            # compute actual arc arrival time
            actual_arrival_time = cond['actual_arrival_time'].iloc[1]
            # compute actual arc travel time
            actual_travel_time = actual_arrival_time - actual_departure_time
            
            # Partial construction of the arc 
            to_concat = cond.iloc[:1].drop(['latitude', 'longitude', 'stop_name'], axis=1)
            to_concat['scheduled_departure_time'] = scheduled_departure_time
            to_concat['scheduled_arrival_time'] = scheduled_arrival_time
            to_concat['scheduled_travel_time'] = scheduled_travel_time
            to_concat['actual_departure_time'] = actual_departure_time
            to_concat['actual_arrival_time'] = actual_arrival_time
            to_concat['actual_travel_time'] = actual_travel_time
            to_concat['origin'] = origin
            to_concat['origin_coord'] = cond['longitude'].iloc[0] + ':' + cond['latitude'].iloc[0]
            to_concat['destination'] = destination
            to_concat['destination_coord'] = cond['longitude'].iloc[1] + ':' + cond['latitude'].iloc[1]
            to_concat = to_concat.reset_index()
            to_concat = to_concat.drop('operator_name', axis=1)
            #to_concat = to_concat.set_index(['origin', 'origin_coord', 'destination', 'destination_coord', 'product_id'])
            to_concat = to_concat.reset_index().drop(['index'], axis=1)
            
            df_partial_edges = pd.concat([df_partial_edges, to_concat])

In [69]:
# change format to datetime
df_partial_edges['actual_travel_time'] = pd.to_datetime(df_partial_edges['actual_travel_time'], format='%d %H:%M:%S', errors='ignore')
# add column travel delay that helps to find parameters
#df_partial_edges['travel_delay [sec]'] = ((df_partial_edges['actual_travel_time'] - df_partial_edges['scheduled_travel_time']) / np.timedelta64(1, 's'))
# reorder columns
df_partial_edges = df_partial_edges[['origin', 'origin_coord', 'destination', 'destination_coord', 'product_id', 'line_id',
                     'scheduled_departure_time', 'scheduled_arrival_time', 'scheduled_travel_time', 
                     'actual_departure_time', 'actual_arrival_time', 'actual_travel_time', 'travel_delay [sec]']]
df_partial_edges = df_partial_edges.reset_index(drop=True)

In [70]:
df_partial_edges.head()

,origin,origin_coord,destination,destination_coord,product_id,line_id,scheduled_departure_time,scheduled_arrival_time,scheduled_travel_time,actual_departure_time,actual_arrival_time,actual_travel_time,travel_delay [sec]
0,Genève,6.142452:46.210203,Vernier,6.093894:46.220710,Zug,11214,2017-09-13 05:18:00,2017-09-13 05:22:00,00:04:00,2017-09-13 05:23:10,2017-09-13 05:27:56,00:04:46,46.0
1,Genève,6.142452:46.210203,Vernier,6.093894:46.220710,Zug,11216,2017-09-13 05:47:00,2017-09-13 05:51:00,00:04:00,2017-09-13 05:53:49,2017-09-13 05:57:56,00:04:07,7.0
2,Genève,6.142452:46.210203,Vernier,6.093894:46.220710,Zug,11220,2017-09-13 06:18:00,2017-09-13 06:21:00,00:03:00,2017-09-13 06:19:27,2017-09-13 06:23:48,00:04:21,81.0
3,Genève,6.142452:46.210203,Vernier,6.093894:46.220710,Zug,11222,2017-09-13 06:47:00,2017-09-13 06:50:00,00:03:00,2017-09-13 06:47:53,2017-09-13 06:51:33,00:03:40,40.0
4,Genève,6.142452:46.210203,Vernier,6.093894:46.220710,Zug,11224,2017-09-13 07:17:00,2017-09-13 07:20:00,00:03:00,2017-09-13 07:19:00,2017-09-13 07:22:53,00:03:53,53.0


In [71]:
df_partial_edges['actual_travel_time'].head()

0   00:04:46
1   00:04:07
2   00:04:21
3   00:03:40
4   00:03:53
Name: actual_travel_time, dtype: timedelta64[ns]

In [81]:
df_partial_edges_viz = df_partial_edges
df_partial_edges_viz = df_partial_edges_viz.set_index(['origin', 'origin_coord', 'destination', 'destination_coord', 'product_id'])
df_partial_edges_viz = df_partial_edges_viz[['scheduled_departure_time', 'scheduled_arrival_time', 'scheduled_travel_time',
                'actual_departure_time', 'actual_arrival_time', 'actual_travel_time', 'travel_delay [sec]']]

In [82]:
df_partial_edges_viz.head()

scheduled_departure_time  \
origin origin_coord       destination destination_coord  product_id                            
Genève 6.142452:46.210203 Vernier     6.093894:46.220710 Zug             2017-09-13 05:18:00   
                                                         Zug             2017-09-13 05:47:00   
                                                         Zug             2017-09-13 06:18:00   
                                                         Zug             2017-09-13 06:47:00   
                                                         Zug             2017-09-13 07:17:00   

                                                                    scheduled_arrival_time  \
origin origin_coord       destination destination_coord  product_id                          
Genève 6.142452:46.210203 Vernier     6.093894:46.220710 Zug           2017-09-13 05:22:00   
                                                         Zug           2017-09-13 05:51:00   
                                                         Zug           2017-09-13 06:21:00   
                                                         Zug           2017-09-13 06:50:00   
                                                         Zug           2017-09-13 07:20:00   

                                                                    scheduled_travel_time  \
origin origin_coord       destination destination_coord  product_id                         
Genève 6.142452:46.210203 Vernier     6.093894:46.220710 Zug                     00:04:00   
                                                         Zug                     00:04:00   
                                                         Zug                     00:03:00   
                                                         Zug                     00:03:00   
                                                         Zug                     00:03:00   

                                                                    actual_departure_time  \
origin origin_coord       destination destination_coord  product_id                         
Genève 6.142452:46.210203 Vernier     6.093894:46.220710 Zug          2017-09-13 05:23:10   
                                                         Zug          2017-09-13 05:53:49   
                                                         Zug          2017-09-13 06:19:27   
                                                         Zug          2017-09-13 06:47:53   
                                                         Zug          2017-09-13 07:19:00   

                                                                    actual_arrival_time  \
origin origin_coord       destination destination_coord  product_id                       
Genève 6.142452:46.210203 Vernier     6.093894:46.220710 Zug        2017-09-13 05:27:56   
                                                         Zug        2017-09-13 05:57:56   
                                                         Zug        2017-09-13 06:23:48   
                                                         Zug        2017-09-13 06:51:33   
                                                         Zug        2017-09-13 07:22:53   

                                                                    actual_travel_time  \
origin origin_coord       destination destination_coord  product_id                      
Genève 6.142452:46.210203 Vernier     6.093894:46.220710 Zug                  00:04:46   
                                                         Zug                  00:04:07   
                                                         Zug                  00:04:21   
                                                         Zug                  00:03:40   
                                                         Zug                  00:03:53   

                                                                     travel_delay [sec]  
origin origin_coord       destination destination_coord  product_id      

In [76]:
# Define time slots
# WARNING those variable names should not be reassigned in the notebook
minuit = datetime.time(0, 0)
midi = datetime.time(12, 0)
defined_timeslots = [str(minuit) + '-' + str(midi), str(midi) + '-' + str(minuit)]

In [88]:
# Utility function to group edge departure times by time slot
def travel_timeslots(index):   
    timestamp = edge_travels.loc[index]['scheduled_departure_time']
    target = datetime.time(timestamp.hour, timestamp.minute, timestamp.second)
    
    if minuit <= target < midi:
        return defined_timeslots[0]
    else: # midi <= target < minuit
        return defined_timeslots[1]

In [89]:
# Create dataframe with necessary edge informations to perform routing
df_edges = pd.DataFrame(columns=['fromNode', 'fromCoord', 'toNode', 'toCoord', 
                                 'product_id', 'timeslots', 'edge_params [µ, s]'])
# List of all edges
edges = df_partial_edges[['origin', 'origin_coord', 'destination', 'destination_coord', 'product_id']].drop_duplicates()

for index, values in edges.iterrows():
        fromNode = values.iloc[0]
        fromCoord = values.iloc[1]
        toNode = values.iloc[2]
        toCoord = values.iloc[3]
        transport = values.iloc[4]
        
        # Get all travel through the edge
        edge_travels = df_partial_edges[((df_partial_edges['origin'] == fromNode) & (df_partial_edges['origin_coord'] == fromCoord) &
                                    (df_partial_edges['destination'] == toNode) & (df_partial_edges['destination_coord'] == toCoord) &
                                    (df_partial_edges['product_id'] == transport))]
        # keep only relevant columns
        edge_travels = edge_travels[['origin', 'origin_coord', 'destination', 'destination_coord', 'product_id', 
                                     'scheduled_departure_time', 'scheduled_travel_time', 'actual_travel_time']]
        # add travel delays for the edge
        edge_travels['travel_delay [sec]'] = (edge_travels['actual_travel_time'] - edge_travels['scheduled_travel_time']) / np.timedelta64(1, 's')
        # Group edges by scheduled departure time slots
        edge_grouped = edge_travels.groupby(travel_timeslots, axis=0)['travel_delay [sec]']
        
        # timeslots with at least one element in the dataframe 'edge_travels'
        observed_timeslots = list(edge_grouped.groups.keys())
        # instantiate parameter list
        edge_params = []
        for name in defined_timeslots:
            if name in observed_timeslots:
                # indices in the dataframe 'edge_travels' associated with the timeslot 'name'
                group_indices = edge_grouped.groups[name]
                # rows in the dataframe 'edge_travels' associated with the timeslot 'name'
                group_elems = edge_travels.loc[group_indices]['travel_delay [sec]']
                # add mean and standard deviation of departure delays for the time slot 'name'
                edge_params.append((round(group_elems.mean(), 1), round(group_elems.std(), 1)))
            else:
                nan = float('NaN')
                edge_params.append((nan, nan))
        
        
        # Generate node information and add it to the dataframe df_nodes
        #print(len(timeslots), len(departure_params), len(arrival_params))
        if len(defined_timeslots) == len(edge_params):
            to_concat = pd.DataFrame({'fromNode': fromNode, 'fromCoord': fromCoord, 'toNode': toNode, 'toCoord': toCoord, 
                                      'product_id': transport, 'timeslots': defined_timeslots, 
                                      'edge_params [µ, s]': edge_params})
            df_edges = pd.concat([df_edges, to_concat], axis=0)        
        '''else:
            check.append(('j=' + str(j), 
                          'timeslots='+str(len(timeslots)), 
                          'edge_params='+str(len(departure_params))))'''

In [94]:
df_edges = df_edges[['fromNode', 'fromCoord', 'toNode', 'toCoord', 'product_id', 'timeslots', 'edge_params [µ, s]']]
df_edges = df_edges.reset_index(drop=True)
df_edges.head()

,fromNode,fromCoord,toNode,toCoord,product_id,timeslots,"edge_params [µ, s]"
0,Genève,6.142452:46.210203,Vernier,6.093894:46.220710,Zug,00:00:00-12:00:00,"(51.1, 19.6)"
1,Genève,6.142452:46.210203,Vernier,6.093894:46.220710,Zug,12:00:00-00:00:00,"(53.9, 10.1)"
2,Vernier,6.093894:46.220710,Meyrin,6.076912:46.222332,Zug,00:00:00-12:00:00,"(37.9, 4.1)"
3,Vernier,6.093894:46.220710,Meyrin,6.076912:46.222332,Zug,12:00:00-00:00:00,"(40.5, 11.9)"
4,Meyrin,6.076912:46.222332,Zimeysa,6.065740:46.221261,Zug,00:00:00-12:00:00,"(-60.0, 0.0)"


# NODES

In [96]:
# Get list of all nodes in the network

# Origin nodes
nodes_1 = df_partial_edges[['origin', 'origin_coord', 'product_id']].drop_duplicates()
nodes_1.columns = ['node', 'coord', 'product_id']
# Destination nodes
nodes_2 = df_partial_edges[['destination', 'destination_coord', 'product_id']].drop_duplicates()
nodes_2.columns = nodes_1.columns

# Construct dataframe of nodes
nodes = pd.concat([nodes_1, nodes_2], axis=0).drop_duplicates().reset_index(drop=True)

In [97]:
nodes.head()

,node,coord,product_id
0,Genève,6.142452:46.210203,Zug
1,Vernier,6.093894:46.220710,Zug
2,Meyrin,6.076912:46.222332,Zug
3,Zimeysa,6.065740:46.221261,Zug
4,Satigny,6.037509:46.214221,Zug


In [98]:
# Utility function to group node departures by time slot
def departure_date_range(index):   
    timestamp = from_node.loc[index]['scheduled_departure_time']
    target = datetime.time(timestamp.hour, timestamp.minute, timestamp.second)
    
    if minuit <= target < midi:
        return defined_timeslots[0]
    else: # midi <= target < minuit:
        return defined_timeslots[1]

In [99]:
# Utility function to group node arrivals by time slot
def arrival_date_range(index):
    timestamp = to_node.loc[index]['scheduled_arrival_time']
    target = datetime.time(timestamp.hour, timestamp.minute, timestamp.second)
    
    if minuit <= target < midi:
        return defined_timeslots[0]
    else: #midi <= target < minuit:
        return defined_timeslots[1]

In [100]:
# Create dataframe with necessary node information to perform routing
df_nodes = pd.DataFrame(columns=['node', 'coord', 'product_id', 'timeslots', 'departure_params [µ, s]', 'arrival_params [µ, s]'])

j = 0
for index, values in nodes.iterrows():
        node = values.iloc[0]
        coord = values.iloc[1]
        transport = values.iloc[2]    
        # Get all edges leaving the current node
        from_node = df_partial_edges[((df_partial_edges['origin'] == node) & (df_partial_edges['origin_coord'] == coord))]
        # Keep only relevant columns
        from_node = from_node[['origin', 'origin_coord', 'destination','destination_coord', 'product_id', 'scheduled_departure_time', 'actual_departure_time']]
        # Add departure delay values for the node 
        from_node['departure_delay [sec]'] = (from_node['actual_departure_time'] - from_node['scheduled_departure_time']) / np.timedelta64(1, 's')
        # Group leaving edges by scheduled departure time slots
        from_grouped = from_node.groupby(departure_date_range, axis=0)['departure_delay [sec]']
        # mean and standard deviation of departure delays for each time slot of the current node
        #departure_params = [(round(group.mean(), 1), round(group.std(), 1)) for _, group in from_grouped]
        
        # timeslots with at least one element in the dataframe 'from_node'
        observed_timeslots = list(from_grouped.groups.keys())
        dep_params = []
        for name in defined_timeslots:
            if name in observed_timeslots:
                # indices in the dataframe 'from_node' associated with the timeslot 'name'
                group_indices = from_grouped.groups[name]
                # rows in the dataframe 'from_node' associated with the timeslot 'name'
                group_elems = from_node.loc[group_indices]['departure_delay [sec]']
                # add mean and standard deviation of departure delays for the time slot 'name'
                dep_params.append((round(group_elems.mean(), 1), round(group_elems.std(), 1)))
            else:
                nan = float('NaN')
                dep_params.append((nan, nan))

        # Get all edges arriving at the current node
        to_node = df_partial_edges[((df_partial_edges['destination'] == node) & (df_partial_edges['destination_coord'] == coord))]
        # Keep only relevant columns
        to_node = to_node[['origin', 'origin_coord', 'destination','destination_coord', 'product_id',  'scheduled_arrival_time', 'actual_arrival_time']]
        # Add arrival delay values for the node
        to_node['arrival_delay [sec]'] = (to_node['actual_arrival_time'] - to_node['scheduled_arrival_time']) / np.timedelta64(1, 's')
        # Group arriving edges by scheduled arrival time
        to_grouped = to_node.groupby(arrival_date_range, axis=0)['arrival_delay [sec]']
        # mean and standard deviation of arrival delays for each time slot of the current node
        # arrival_params = [(round(group.mean(), 1), round(group.std(), 1)) for _, group in to_grouped]
        
        # timeslots with at least one element in the dataframe 'to_node'
        observed_timeslots = list(to_grouped.groups.keys())
        arr_params = []
        for name in defined_timeslots:
            if name in observed_timeslots:
                # indices in the dataframe 'from_node' associated with the timeslot 'name'
                group_indices = to_grouped.groups[name]
                # rows in the dataframe 'from_node' associated with the timeslot 'name'
                group_elems = to_node.loc[group_indices]['arrival_delay [sec]']
                # add mean and standard deviation of departure delays for the time slot 'name'
                arr_params.append((round(group_elems.mean(), 1), round(group_elems.std(), 1)))
            else:
                nan = float('NaN')
                arr_params.append((nan, nan))


        # Generate node information and add it to the dataframe df_nodes
        #print(len(timeslots), len(departure_params), len(arrival_params))
        if len(defined_timeslots) == len(dep_params) == len(arr_params):
            to_concat = pd.DataFrame({'node': node, 'coord': coord, 'product_id': transport, 'timeslot': defined_timeslots, 
                                   'departure_params [µ, s]': dep_params, 'arrival_params [µ, s]': arr_params})
            df_nodes = pd.concat([df_nodes, to_concat], axis=0)
        '''else:
            check.append(('j=' + str(j), 
                          'timeslots='+str(len(timeslots)), 
                          'departure_params='+str(len(departure_params)), 
                          'arrival_params='+str(len(arrival_params))))'''

In [101]:
df_nodes = df_nodes[['node', 'coord', 'product_id', 'timeslot', 'departure_params [µ, s]', 'arrival_params [µ, s]']]
df_nodes = df_nodes.reset_index(drop=True)
df_nodes.head()

,node,coord,product_id,timeslot,"departure_params [µ, s]","arrival_params [µ, s]"
0,Genève,6.142452:46.210203,Zug,00:00:00-12:00:00,"(168.7, 127.0)","(84.5, 116.5)"
1,Genève,6.142452:46.210203,Zug,12:00:00-00:00:00,"(89.8, 56.8)","(48.6, 90.6)"
2,Vernier,6.093894:46.220710,Zug,00:00:00-12:00:00,"(184.0, 95.9)","(117.9, 97.7)"
3,Vernier,6.093894:46.220710,Zug,12:00:00-00:00:00,"(163.6, 71.7)","(93.6, 79.0)"
4,Meyrin,6.076912:46.222332,Zug,00:00:00-12:00:00,"(220.6, 106.5)","(192.1, 102.7)"
